# Part 2, Lab #1: Image Classification

Ke Xu 3190110360

### Due March 29th, 2023 11:59 PM CST

#### Logistics and Lab Submission

See the **BlackBoard**.

#### What You Will Need To Know For This Lab

This lab covers:

- Design the dataset.
- Design the convolutional neural network for the image classification.
- Training and testing the model.

The submission procedure is provided below:
- You will be provided with a Jupyter Notebook for this lab where you need to implement the provided functions as needed for each question. Follow the instructions provided in this Jupyter Notebook (.ipynb) to implement the required functions. 
- Upload the **PDF** (screen shot) file of your Jupyter Notebook (.ipynb file).
- Your grades and feedbacks will appear on BlackBoard. You will have a chance to re-submit your code, only if you have *reasonable* submissions before the deadline (i.e. not an empty script).

# Problem 1 : Install packages(15 points)

In this section, you will install PIL, pytorch packages. You should follow these commands to install and print results. Note that, we only use pytorch in CPU version. If you want to install pytorch in GPU version, please install from [pytorch.org](https://pytorch.org/get-started/locally/).

In [1]:
!pip install torch
!pip install pillow

Use torch to generate a zero matrix in shape (2,3). load and print the size of image "test.jpg".

In [7]:
import torch
import numpy as np
from PIL import Image
print("Zero Tensor:")
print(torch.zeros(2, 3))

test_img = np.array(Image.open('test.jpg'))
print("image shape is:",test_img.shape)

Zero Tensor:
tensor([[0., 0., 0.],
        [0., 0., 0.]])
image shape is: (533, 800, 3)


## Problem 2: Define dataset (15 points)

Desing a **Dataloader** for the dataset. Print the 10th image size and label. Note that batch size is 1.

In [3]:
from torch.utils.data import Dataset, DataLoader
#import numpy as np
#from PIL import Image
import os
from torchvision import transforms
from torchvision.utils import make_grid


class MyData(Dataset):

    def __init__(self, root_dir, image_dir, label_dir, transform):
        self.root_dir = root_dir
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.label_path = os.path.join(self.root_dir, self.label_dir)
        self.image_path = os.path.join(self.root_dir, self.image_dir)
        self.image_list = os.listdir(self.image_path)
        self.label_list = os.listdir(self.label_path)
        self.transform = transform
        self.image_list.sort()
        self.label_list.sort()

    def __getitem__(self, idx):
        img_name = self.image_list[idx]
        label_name = self.label_list[idx]
        img_item_path = os.path.join(self.root_dir, self.image_dir, img_name)
        label_item_path = os.path.join(self.root_dir, self.label_dir, label_name)
        img = Image.open(img_item_path)

        with open(label_item_path, 'r') as f:
            label = f.readline()

        img = self.transform(img)
        sample = {'img': img, 'label': label}
        return sample

    def __len__(self):
        assert len(self.image_list) == len(self.label_list)
        return len(self.image_list)

if __name__ == '__main__':
    #transform
    transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])

    #dataset
    root_dir = "./dataset/train"
    image_bees = "bees_image"
    label_bees = "bees_label"
    bees_dataset = MyData(root_dir, image_bees, label_bees, transform)
    train_dataset = bees_dataset

    #dataloader
    dataloader = DataLoader(train_dataset, batch_size=1, num_workers=0,shuffle = True)
    
    for ind, data in enumerate(dataloader):
        if ind == 9:
            print("This is 10th image")
            print("img shape: ",data['img'].shape)
            print("label: ",data['label'])

This is 10th image
img shape:  torch.Size([1, 3, 256, 256])
label:  ['bees']


## Problem 3: Design an image classification network (70 points)

In this problem, you should build a image classifier by design the network, training the network and testing the model. The dataset is Cifar-10.

### Problem 3-1: Design a network and print the structure as shown in the output. (20 points)

In [4]:
import torch
from torch import nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # you should initialize the Net
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4, 64),
            nn.Linear(64, 10)
        )
        
    def forward(self, x):
        # you should define the forward
        output = self.model(x)
        return output

if __name__ == '__main__':
    net = Net()
    print(net)

Net(
  (model): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=1024, out_features=64, bias=True)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)


### Problem 3-2: Training the network (25 points)

In this part, you should train your network in Problem 3-1. The loss function is Cross Entropy loss, learning rate is 1e-2, epoch is 10, optimizer is SGD. Please print the loss of each epoch and save the final model as "./model/model.pth".

In [5]:
import torch
import torchvision

from torch import nn
from torch.utils.data import DataLoader

# use GPU or CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# set random seed
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     torch.backends.cudnn.deterministic = True
setup_seed(20)

# training dataset
train_data = torchvision.datasets.CIFAR10(root="./data", train=True, transform=torchvision.transforms.ToTensor(),
                                         download=True)
                                         
# Build Dataloader, batch_size is 64
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)

# you should design the training process

net = Net()

loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-2
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)
epoch = 10

print('The size of training set is:',len(train_data))

for i in range(epoch):
    print(f"-------{i+1} epoch starting-------")

    #net.train()
    for data in train_dataloader:
        imgs, targets = data
        outputs = net(imgs)
        loss = loss_fn(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("epoch：{}, Loss: {}".format(i+1, loss.item()))

print('Model saving')
torch.save(net, "./model/model.pth")
print('Finish!')



Files already downloaded and verified
The size of training set is: 50000
-------1 epoch starting-------
epoch：1, Loss: 1.8112249374389648
-------2 epoch starting-------
epoch：2, Loss: 1.5830020904541016
-------3 epoch starting-------
epoch：3, Loss: 1.7640626430511475
-------4 epoch starting-------
epoch：4, Loss: 1.835424780845642
-------5 epoch starting-------
epoch：5, Loss: 1.0777113437652588
-------6 epoch starting-------
epoch：6, Loss: 1.0899240970611572
-------7 epoch starting-------
epoch：7, Loss: 1.5634037256240845
-------8 epoch starting-------
epoch：8, Loss: 1.185918927192688
-------9 epoch starting-------
epoch：9, Loss: 1.1509898900985718
-------10 epoch starting-------
epoch：10, Loss: 1.1384997367858887
Model saving
Finish!


### Problem 3-3: Test on the testing set. (25 points)

Load the testing set and print the accuracy. The accuracy is the average accuracy of all samples.

In [6]:
import torch
import torchvision

#from model import *

from torch import nn
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test_data = torchvision.datasets.CIFAR10(root="./data", train=False, transform=torchvision.transforms.ToTensor(),download=True)
test_data_size = len(test_data)                                         
test_dataloader = DataLoader(test_data, batch_size=64)

# you should design the testing process

#net.eval()
net = torch.load("./model/model.pth")
total_test_loss = 0
total_accuracy = 0

with torch.no_grad():
    for data in test_dataloader:
        imgs, targets = data
        outputs = net(imgs)
        loss = loss_fn(outputs, targets)
        total_test_loss = total_test_loss + loss.item()
        accuracy = (outputs.argmax(1) == targets).sum()
        total_accuracy = total_accuracy + accuracy

print("test accuracy: {}".format(total_accuracy/test_data_size))



Files already downloaded and verified
test accuracy: 0.4659000039100647


## And this concludes Lab 1! Congratulations!